### 数据预处理

现实的数据往往是充满噪声的，⽽没有⾼质量的数据，就没有⾼质量的数据挖掘结果。所以，我们需要对数据进⾏预处理，以提⾼数据的质量。  
数据的质量涉及许多因素，包括：准确性、完整性、⼀致性、时效性、可信性、可解释性

数据预处理的主要步骤为：
- 数据清理：通过填写缺失值、光滑噪声数据、识别或删除离群点。并解决不⼀致性来“清理”数据
- 数据集成：将多个数据源、数据库集成为⼀个
- 数据规约：将得到的数据进⾏简化，去除冗余数据
- 数据变换：讲数据进⾏规范化、数据离散化和数据分层，可以使得数据挖掘在多个抽象层次上进⾏。

[sklearn.preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

#### 数据编码(encoding)

对离散数据进行编码的总原则：
- 离散特征的取值之间没有大小的意义，比如color: [red, blue]，性别的男女等，那么就使用OneHot编码
- 离散特征的取值有大小的意义，比如size: [X,XL,XXL]，身高的高、中、低等，那么就使用数值的映射{X:1,XL:2,XXL:3}进行编码


In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({'婚姻状况':['单身', '已婚', '单身', '已婚', '离婚'], 
                     '是否拥有房产':['是', '否', '否', '是', '否'], 
                     '年收入':[12.5, 10, 7, 12, 9.5],
                     '是否能偿还债务':['否', '否', '否', '否', '是']})
data

,婚姻状况,是否拥有房产,年收入,是否能偿还债务
0,单身,是,12.5,否
1,已婚,否,10.0,否
2,单身,否,7.0,否
3,已婚,是,12.0,否
4,离婚,否,9.5,是


`sklearn.preprocessing.LabelEncoder`

>使用0到n_classes-1之间的值对目标标签进行编码。该转换器应用于编码**目标值**（接受一维数组），即 y，而不是输入X。

In [2]:
from sklearn.preprocessing import LabelEncoder

LabelEncoder().fit_transform(data['是否能偿还债务'])

array([0, 0, 0, 0, 1])

`sklearn.preprocessing.OrdinalEncoder`

> 将分类**特征**编码为整数数组（接受二维数组）。  
该转换器的输入应为整数或字符串之类的数组，表示分类（离散）特征所采用的值。特征值将转换为0到n_categories-1的整数序列。

In [3]:
from sklearn.preprocessing import OrdinalEncoder

OrdinalEncoder().fit_transform(data[['婚姻状况', '是否拥有房产']])

array([[0., 1.],
       [1., 0.],
       [0., 0.],
       [1., 1.],
       [2., 0.]])

**独热编码**

可以这样理解，对于每一个特征，如果它有m个可能值，那么经过独热编码后，就变成了m个**二元**特征（如成绩这个特征有[好，中，差]变成one-hot就是[100, 010, 001]）。并且，这些特征互斥，每次只有一个激活。因此，数据会变成稀疏的。

这样做的好处主要有：
- 解决了分类器不好处理属性数据的问题 
- 在一定程度上也起到了扩充特征的作用

采取one-hot编码是因为大部分算法是基于向量空间中的度量来进行计算的，目的是为了使非偏序关系的变量取值不具有偏序性，并且到原点是等距的。将离散型特征使用one-hot编码，会让特征之间的距离计算更加合理。

方法：
- `pd.get_dummies`  
- `sklearn.preprocessing.OneHotEncoder`

In [4]:
data

,婚姻状况,是否拥有房产,年收入,是否能偿还债务
0,单身,是,12.5,否
1,已婚,否,10.0,否
2,单身,否,7.0,否
3,已婚,是,12.0,否
4,离婚,否,9.5,是


In [5]:
pd.get_dummies(data[['婚姻状况', '是否拥有房产']], sparse=True)

,婚姻状况_单身,婚姻状况_已婚,婚姻状况_离婚,是否拥有房产_否,是否拥有房产_是
0,1,0,0,0,1
1,0,1,0,1,0
2,1,0,0,1,0
3,0,1,0,0,1
4,0,0,1,1,0


In [6]:
from sklearn.preprocessing import OneHotEncoder

print(OneHotEncoder().fit_transform(data[['婚姻状况', '是否拥有房产']]).todense())

[[1. 0. 0. 0. 1.]
 [0. 1. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 0.]]


#### 数据清洗(cleaning)
现实中的数据⼀般是不完整的、有噪声的和不⼀致的。数据清洗试图填充缺失值、光滑噪声并识别离群点和纠正数据中的不⼀致。

有时候我们获取的数据存在**缺失值**，往往⽤NaN来表示。  
忽略缺失值：当缺失值较少的时候，我们可以丢弃缺失的元组，⽽缺失值较多的时候，我们需要采取别的⽅法。

**基于pandas填补缺失值**
- 判断缺失值：isna/notna
- 处理缺失值：dropna/fillna

#### 数据离散化

离散化 (Discretization) (有些时候叫 量化(quantization) 或 分箱(binning)) 提供了将连续特征划分为离散特征值的方法。 某些具有连续特征的数据集会受益于离散化，因为离散化可以把具有连续属性的数据集变换成只有标称属性(nominal attributes)的数据集。

**K-bins离散化**

`sklearn.preprocessing.KBinsDiscretizer`类使用k个bins把特征离散化，默认情况下返回one-hot编码后的稀疏矩阵，可以使用encode参数进行配置。

参数：
- n_bins：每个特征中分箱的个数，默认为5
- encode：编码方式，可取值为{'onehot', 'ordinal', 'onehot-dense'}，默认为'onehot'
- strategy：定义分箱宽度的方式，可取值为{'quantile':等频分箱, 'uniform':等宽分箱, 'k-means':按聚类分箱，每个箱中的样本到最近的一维k均值聚类簇心的距离都相等}，默认为'quantile'

In [7]:
import pandas as pd
import numpy as np

data = pd.DataFrame({'婚姻状况':['单身', '已婚', '单身', '已婚', '离婚'], 
                     '是否拥有房产':['是', '否', '否', '是', '否'], 
                     '年收入':[12.5, 10, 7, 12, 9.5],
                     '年龄':[20, 28, 35, 23, 46],
                     '是否能偿还债务':['否', '否', '否', '否', '是']})
data

,婚姻状况,是否拥有房产,年收入,年龄,是否能偿还债务
0,单身,是,12.5,20,否
1,已婚,否,10.0,28,否
2,单身,否,7.0,35,否
3,已婚,是,12.0,23,否
4,离婚,否,9.5,46,是


In [8]:
from sklearn.preprocessing import KBinsDiscretizer

KBinsDiscretizer(n_bins=[2, 3], encode = 'ordinal', strategy = 'uniform').fit_transform(data[['年收入', '年龄']])

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 2.]])

**特征二值化**

`sklearn.preprocessing.Binarizer`类根据阈值将数据二值化（将特征值设置为0或1），用于处理连续型变量。大于阈值的值映射为1，而小于或等于阈值的值映射为0。默认阈值为0时，特征中所有的正值都映射到1。二值化是对文本计数数据的常见操作，分析人员可以决定仅考虑某种现象的存在与否。

参数：
- threshold：用来划分原始数据特征的阈值，默认值为0

In [9]:
from sklearn.preprocessing import Binarizer

Binarizer(threshold=[10, 30]).fit_transform(data[['年收入', '年龄']])

array([[1., 0.],
       [0., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

### 特征选择 - Filter方法

`sklearn.feature_selection`

In [10]:
from sklearn.datasets import load_iris

#导入IRIS数据集
iris = load_iris()

#特征矩阵
iris.data

#目标向量
# iris.target

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

**方差选择法**

使用方差选择法，先要计算各个特征的方差，然后根据阈值，选择方差大于阈值的特征。使用feature_selection库的VarianceThreshold类来选择特征的代码如下：

In [11]:
from sklearn.feature_selection import VarianceThreshold

#参数threshold为方差的阈值
X = VarianceThreshold(threshold=3).fit_transform(iris.data)
X.shape

(150, 1)

**相关系数法**

皮尔森相关系数是一种最简单的，能帮助理解特征和响应变量之间关系的方法，该方法衡量的是变量之间的线性相关性，结果的取值区间为[-1，1]，-1表示完全的负相关，+1表示完全的正相关，0表示没有线性相关。

使用相关系数法，需要计算各个特征对目标值的相关系数以及相关系数的P值。

Scipy的 pearsonr 方法能够同时计算 相关系数 和p-value.

下面这个例子中，我们比较了变量在加入噪音之前和之后的差异。当噪音比较小的时候，相关性很强，p-value很低。

In [12]:
import numpy as np
from scipy.stats import pearsonr
np.random.seed(0)
size = 300
x = np.random.normal(0, 1, size)
# pearsonr(x, y)的输入为特征矩阵和目标向量
print("Lower noise：", pearsonr(x, x + np.random.normal(0, 1, size)))
print("Higher noise：", pearsonr(x, x + np.random.normal(0, 10, size)))

Lower noise： (0.7182483686213833, 7.324017313001002e-49)
Higher noise： (0.057964292079338064, 0.3170099388532581)


Pearson相关系数的一个明显缺陷是，作为特征排序机制，他只对线性关系敏感。如果关系是非线性的，即便两个变量具有一一对应的关系，Pearson相关性也可能会接近0。

**卡方检验法**

经典的卡方检验是检验定性自变量对定性因变量的相关性。假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距，构建统计量：

${\chi}^2 = \Sigma \frac{(A-E)^2}{E}$

这个统计量的含义简而言之就是自变量对因变量的相关性。用feature_selection库的SelectKBest类结合卡方检验来选择特征的代码如下：


In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
 
#选择K个最好的特征，返回选择特征后的数据
X = SelectKBest(chi2, k=2).fit_transform(iris.data, iris.target)
X.shape

(150, 2)